In [15]:
import numpy as np
from scipy.stats import norm, t
from scipy.integrate import quad

# Parameters (keeping original settings)
gamma = 1e-8
beta = 10**8
lambda_init = 1e-4
n_iterations = 10**6
n_samples = 10000
burn_in_samples = 10000

# Quantile levels
q_levels = [0.95, 0.99]

# Normal distribution parameters
normal_params = [(0, 1), (1, 2), (3, 5)]

# Student's t distribution parameters
t_params = [10, 7, 3]

def calculate_var_cvar(x_samples, q_bar):
    theta_values = np.zeros(burn_in_samples)
    theta = 0.0
    
    for i in range(n_iterations):
        x = np.random.choice(x_samples)
        # Gradient calculation
        gradient = 1 - (1.0 / (1.0 - q_bar)) * (1 if x >= theta else 0) + 2 * gamma * theta
        noise = np.random.normal(0, 1)
        
        # Theta update
        theta -= lambda_init * gradient + np.sqrt(2 * lambda_init / beta) * noise

        # Collect last samples for theta
        if i >= n_iterations - burn_in_samples:
            index = i - (n_iterations - burn_in_samples)
            theta_values[index] = theta
    
    var = np.mean(theta_values)
    
    # Calculate CVaR
    excess_loss = [max(x - var, 0) for x in x_samples]
    cvar = var + np.mean(excess_loss) / (1 - q_bar) + gamma * theta**2
    
    return var, cvar

def theoretical_var_cvar_normal(mu, sigma, q_bar):
    var = norm.ppf(q_bar, loc=mu, scale=sigma)
    cvar = mu + (sigma * norm.pdf(norm.ppf(q_bar)) / (1 - q_bar))
    return var, cvar

def theoretical_var_cvar_student_t(df, q_bar):
    var = t.ppf(q_bar, df)
    
    # CVaR calculation using integration for t-distribution
    def t_cvar_integrand(x):
        return x * t.pdf(x, df)
    
    cvar_integral, _ = quad(t_cvar_integrand, var, np.inf)
    cvar = cvar_integral / (1 - q_bar)
    
    return var, cvar

def main():
    results = []

    for q_bar in q_levels:
        # Normal distribution calculations
        for mu, sigma in normal_params:
            x_normal = np.random.normal(mu, sigma, n_samples)
            var_sgld, cvar_sgld = calculate_var_cvar(x_normal, q_bar)
            var_theory, cvar_theory = theoretical_var_cvar_normal(mu, sigma, q_bar)

            results.append(
                f"Normal Distribution (mu={mu}, sigma={sigma}, q_bar={q_bar}):\n"
                f"  VaR*: {var_theory:.3f}, CVaR*: {cvar_theory:.3f}\n"
                f"  VaRSGLD: {var_sgld:.3f}, CVaRSGLD: {cvar_sgld:.3f}\n"
            )

        # Student's t distribution calculations
        for df in t_params:
            x_t = t.rvs(df, size=n_samples)
            var_sgld, cvar_sgld = calculate_var_cvar(x_t, q_bar)
            var_theory, cvar_theory = theoretical_var_cvar_student_t(df, q_bar)

            results.append(
                f"T Distribution (d.f.={df}, q_bar={q_bar}):\n"
                f"  VaR*: {var_theory:.3f}, CVaR*: {cvar_theory:.3f}\n"
                f"  VaRSGLD: {var_sgld:.3f}, CVaRSGLD: {cvar_sgld:.3f}\n"
            )

    for result in results:
        print(result)

if __name__ == "__main__":
    np.random.seed(0)
    main()


Normal Distribution (mu=0, sigma=1, q_bar=0.95):
  VaR*: 1.645, CVaR*: 2.063
  VaRSGLD: 1.632, CVaRSGLD: 2.033

Normal Distribution (mu=1, sigma=2, q_bar=0.95):
  VaR*: 4.290, CVaR*: 5.125
  VaRSGLD: 4.149, CVaRSGLD: 5.079

Normal Distribution (mu=3, sigma=5, q_bar=0.95):
  VaR*: 11.224, CVaR*: 13.314
  VaRSGLD: 11.392, CVaRSGLD: 13.344

T Distribution (d.f.=10, q_bar=0.95):
  VaR*: 1.812, CVaR*: 2.408
  VaRSGLD: 1.782, CVaRSGLD: 2.382

T Distribution (d.f.=7, q_bar=0.95):
  VaR*: 1.895, CVaR*: 2.595
  VaRSGLD: 1.983, CVaRSGLD: 2.680

T Distribution (d.f.=3, q_bar=0.95):
  VaR*: 2.353, CVaR*: 3.874
  VaRSGLD: 2.397, CVaRSGLD: 4.034

Normal Distribution (mu=0, sigma=1, q_bar=0.99):
  VaR*: 2.326, CVaR*: 2.665
  VaRSGLD: 2.328, CVaRSGLD: 2.634

Normal Distribution (mu=1, sigma=2, q_bar=0.99):
  VaR*: 5.653, CVaR*: 6.330
  VaRSGLD: 5.731, CVaRSGLD: 6.431

Normal Distribution (mu=3, sigma=5, q_bar=0.99):
  VaR*: 14.632, CVaR*: 16.326
  VaRSGLD: 14.448, CVaRSGLD: 16.330

T Distribution (d.f